# Create Driver Dimension

In [0]:
import pyspark.sql.functions as F
from random import randint
from pyspark.sql.types import * 

In [0]:
# Azure Blob Storage details
storage_account_name = "<storage-account-name>"
storage_account_key = "<storage-account-access-key>"
container_name = "deltalake"
file_path1 = "bronze/Trip_Transactions"
file_path2 = "bronze/Rewards_Points"

In [0]:
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

In [0]:
df1=spark.read.format("delta").load(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path1}")

In [0]:
df1.show()

+-------+--------------------+--------------------+---------+-----------+------------------------+-----------+---------------------+--------------+-----------------------------+----------------+--------------------------+-------------------+--------------+----------+-----------+---------------------+--------------------+--------------------+-----------+-------------+
|trip_id|trip_start_timestamp|  trip_end_timestamp|driver_id|driver_name|source_location_address1|source_city|source_province_state|source_country|destination_location_address1|destination_city|destination_province_state|destination_country|total_distance|total_fare|trip_status|delay_start_time_mins|      payment_method|      payment_Status|customer_id|customer_name|
+-------+--------------------+--------------------+---------+-----------+------------------------+-----------+---------------------+--------------+-----------------------------+----------------+--------------------------+-------------------+--------------+----

In [0]:
df1.createOrReplaceTempView("df1") 

In [0]:
df2=spark.sql('select distinct driver_id,driver_name from df1')

In [0]:
def driver_age(driver_id):
    return randint(18,65)
def driver_gender(driver_id):
    if randint(0,2)==0:
        return 'F'
    else:
        return "M"
    

In [0]:
spark.udf.register("driver_age", driver_age,IntegerType())
spark.udf.register("driver_gender", driver_gender,StringType())

Out[22]: <function __main__.driver_gender(driver_id)>

In [0]:
df2=df2.withColumn("driver_age",F.expr("driver_age(driver_id)"))
df2=df2.withColumn("driver_gender",F.expr("driver_gender(driver_id)"))



In [0]:
display(df2)

driver_id,driver_name,driver_age,driver_gender
A33,Umesh,18,F
A10,Amith,60,M
A23,Dhruv,43,M
A5,Sagar,47,M
A40,Ramesh,58,M
A1,Ram,38,M
A4,Aaditya,60,M
A28,Jack,57,M
A21,Arun,53,M
null,null,43,M


In [0]:
file_path6 = "silver_Zone/driver_Dimension"

In [0]:
df2.repartition(1).write.save(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path6}")